In [1]:
import numpy as np
import pandas as pd

# Numpy vs Pandas

Un aspecto al que muchas personas no prestan suficiente atención, lo que a menudo conduce a un uso ineficiente de Pandas, es que esta biblioteca está construida en torno al formato de columnas.

**Pandas** se basa en el objeto `DataFrame`, un concepto inspirado en el Data Frame de **R**, que privilegia la organización por columnas. Un `DataFrame` es una tabla bidimensional compuesta por filas y columnas.

En **NumPy**, se puede especificar el orden principal de almacenamiento. Al crear un `ndarray`, este es de fila principal de forma predeterminada, a menos que se especifique otro orden. Dado que **Pandas** utiliza **NumPy** "por debajo" (como su base), a veces se comete el error de tratar los `DataFrame` de la misma manera que se haría con los `ndarray`. Por ejemplo, intentar acceder a los datos por filas en un `DataFrame` para entrenar un modelo puede generar un procesamiento innecesariamente lento.

In [2]:
array = np.random.randint(1, 9001, size=(9000, 10))
n_rows, n_cols = array.shape
df = pd.DataFrame(array, columns=[i for i in "abcdefghik"])

En la siguientes celdas, se puede ver que acceder a un `DataFrame` por fila es mucho más lento que acceder al mismo `DataFrame` por columna.


In [3]:
%%timeit
# Iterando un DataFrame por columna
for col in df.columns:
    for item in df[col]:
        pass

4.41 ms ± 298 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%%timeit
# Iterando un DataFrame por fila
n_rows = len(df)
for i in range(n_rows):
    for item in df.iloc[i]:
        pass

94 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
time_col = 4.28
time_row = 79.7
print("Hacer una iteración por fila lleva un " + str(round((time_row-time_col) * 100 / time_col, 0)) + "% mas lento que por columna")

Hacer una iteración por fila lleva un 1762.0% mas lento que por columna


En cambio si vemos directamente desde el `ndarray` (vamos a usar el método `flatten` para poder notar el efecto), pero primero obtenemos un array en modo columna principal (dado qeu podemos que en Numpy podemos elegir este comportamineto):

In [6]:
print('Array en orden C (row-major):')
print(' - C_CONTIGUOUS:', array.flags['C_CONTIGUOUS'])
print(' - F_CONTIGUOUS:', array.flags['F_CONTIGUOUS'])

arr_f_copy = array.copy(order='F')
print('Array en orden F (column-major):')
print(' - C_CONTIGUOUS:', arr_f_copy.flags['C_CONTIGUOUS'])
print(' - F_CONTIGUOUS:', arr_f_copy.flags['F_CONTIGUOUS'])

Array en orden C (row-major):
 - C_CONTIGUOUS: True
 - F_CONTIGUOUS: False
Array en orden F (column-major):
 - C_CONTIGUOUS: False
 - F_CONTIGUOUS: True


In [7]:
%%timeit
_ = array.flatten(order='C') # Aplanamos en el sentido de las filas

18.2 μs ± 1.46 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [8]:
%%timeit
_ = array.flatten(order='F') # Aplanamos en el sentido de las columnas

79 μs ± 7.79 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
time_row = 16.2
time_col = 67.3
print("Hacer una iteración por columna lleva un " + str(round((time_col-time_row) * 100 / time_row, 0)) + "% mas lento que por fila")

Hacer una iteración por columna lleva un 315.0% mas lento que por fila


En cambio si vemos en el array que está en modo columna principal:

In [10]:
%%timeit
_ = arr_f_copy.flatten(order='C') # Aplanamos en el sentido de las filas

59.6 μs ± 896 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [11]:
%%timeit
_ = arr_f_copy.flatten(order='F') # Aplanamos en el sentido de las columnas

17.8 μs ± 1.61 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [12]:
time_col = 15.2
time_row = 59.8
print("Hacer una iteración por fila lleva un " + str(round((time_row-time_col) * 100 / time_col, 0)) + "% mas lento que por columna")

Hacer una iteración por fila lleva un 293.0% mas lento que por columna


----

## CSV vs Parquet

Cuando se almacena en formato CSV, el archivo de wine ocupa 101 KB. Pero cuando se almacena en Parquet, el mismo archivo ocupa 34 KB.

In [13]:
import os
csv_size = os.path.getsize("./winequality-red.csv")
parquet_size = os.path.getsize("./winequality-red.parquet")

In [14]:
print("El tamaño del CSV es", csv_size, "Bytes")
print("El tamaño del parquet es", parquet_size, "Bytes")
print("El tamaño del parquet es un", (csv_size-parquet_size) * 100 / csv_size, "% mas chico que el csv")

El tamaño del CSV es 100951 Bytes
El tamaño del parquet es 34359 Bytes
El tamaño del parquet es un 65.96467593188775 % mas chico que el csv
